# Creating & Uploading python distribution
https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container

In [59]:
!python setup.py sdist

running sdist
running egg_info
writing dat_package.egg-info\PKG-INFO
writing dependency_links to dat_package.egg-info\dependency_links.txt
writing entry points to dat_package.egg-info\entry_points.txt
writing requirements to dat_package.egg-info\requires.txt
writing top-level names to dat_package.egg-info\top_level.txt
reading manifest file 'dat_package.egg-info\SOURCES.txt'
writing manifest file 'dat_package.egg-info\SOURCES.txt'
running check
creating dat-package-0.1
creating dat-package-0.1\dat_package.egg-info
creating dat-package-0.1\trainer
creating dat-package-0.1\trainer\dat
copying files to dat-package-0.1...
copying README.md -> dat-package-0.1
copying setup.py -> dat-package-0.1
copying dat_package.egg-info\PKG-INFO -> dat-package-0.1\dat_package.egg-info
copying dat_package.egg-info\SOURCES.txt -> dat-package-0.1\dat_package.egg-info
copying dat_package.egg-info\dependency_links.txt -> dat-package-0.1\dat_package.egg-info
copying dat_package.egg-info\entry_points.txt -> dat

In [61]:
!pip install ./dist/dat-package-0.1.tar.gz

Processing c:\users\caspa\documents\01_softwaredevelopmentlocation\vs code\dat\dist\dat-package-0.1.tar.gz
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for dat-package: filename=dat_package-0.1-py3-none-any.whl size=19074 sha256=2a72493f7e22f59698a7bb13df8d8f35801e698307f99988c4fbee1aa4abd5ae
  Stored in directory: c:\users\caspa\appdata\local\pip\cache\wheels\e9\cf\1a\e62844288287d72c2cd43d7db36fd088a5cbebc9a084ce6e96
Successfully built dat-package
  Attempting uninstall: dat-package
    Found existing installation: dat-package 0.1
    Uninstalling dat-package-0.1:
      Successfully uninstalled dat-package-0.1


In [53]:
!python -m trainer.task

c:\Users\caspa\miniconda3\envs\seminar_dml_dat\python.exe: Error while finding module specification for 'trainer.task' (ModuleNotFoundError: No module named 'trainer')


In [44]:
!pip list

Package                       Version
----------------------------- -----------
anyio                         3.6.2
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
asttokens                     2.2.1
attrs                         23.1.0
backcall                      0.2.0
beautifulsoup4                4.12.2
bleach                        6.0.0
cachetools                    5.3.0
certifi                       2023.5.7
cffi                          1.15.1
charset-normalizer            3.1.0
colorama                      0.4.6
comm                          0.1.3
dat-package                   0.1
debugpy                       1.6.7
decorator                     5.1.1
defusedxml                    0.7.1
executing                     1.2.0
fastjsonschema                2.16.3
filelock                      3.12.0
fqdn                          1.5.1
google-api-core               2.11.0
google-auth                   2.18.1
google-clo

In [63]:
#!gcloud storage cp dist/dat_package-0.1.tar.gz gs://dat-package-bucket
!gsutil cp dist/dat-package-0.1.tar.gz gs://dat-project-bucket/dat-package/

Copying file://dist\dat-package-0.1.tar.gz [Content-Type=application/x-tar]...
/ [0 files][    0.0 B/ 14.8 KiB]                                                
/ [1 files][ 14.8 KiB/ 14.8 KiB]                                                

Operation completed over 1 objects/14.8 KiB.                                     


# Creating CustomJob
https://cloud.google.com/vertex-ai/docs/training/create-custom-job#create_custom_job-gcloud

# Operationalize Distributed Training with PyTorch on Google Cloud
https://www.youtube.com/watch?v=kXbDEmn2XOw


Pre-built-containers:
https://cloud.google.com/vertex-ai/docs/training/pre-built-containers#pytorch

GPUs:
https://cloud.google.com/compute/docs/gpus
https://cloud.google.com/compute/gpus-pricing
https://cloud.google.com/vertex-ai/docs/training/configure-compute#gpu-compatibility-table

In [ ]:
#!pip install google-cloud-aiplatform

In [64]:
from google.cloud import aiplatform
from google.cloud.aiplatform_v1.types import job_state
from google.cloud.aiplatform_v1 import types
from google.protobuf import json_format

project_id = 'sdml-dat'
region = 'us-central1' #TODO: change to better region
bucket_name = 'dat-project-bucket'
package_uri = f'gs://{bucket_name}/dat-package/dat-package-0.1.tar.gz'

# Initialize Vertex AI SDK
aiplatform.init(project=project_id, location=region, staging_bucket=bucket_name)

# Define worker pool spec
worker_pool_specs =  [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
        },
        "replica_count": 1,  # Number of masters
        "python_package_spec": {
            "executor_image_uri": "europe-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-13.py310:latest",
            "package_uris": [package_uri],
            "python_module": "trainer.task",
            "args": [],
        },
    }
    # ,
    # {
    #     "machine_spec": {
    #         "machine_type": "n1-standard-4",
    #         "accelerator_type": "NVIDIA_TESLA_K80",
    #         "accelerator_count": 1,
    #     },
    #     "replica_count": 2,  # Number of workers
    #     "python_package_spec": {
    #         "executor_image_uri": "europe-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-13.py310:latest",
    #         "package_uris": [package_uri],
    #         "python_module": "dat_package",
    #         "args": [],
    #     },
    # },
]

# Define job name
job_name = 'dat-custom-pytorch-job'

# Create a custom job
job = aiplatform.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
)

# Launch the job
job.run(sync=True)

print(f"Job {job_name} has been launched.")

Creating CustomJob
CustomJob created. Resource name: projects/65830871515/locations/us-central1/customJobs/8437305747951321088
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/65830871515/locations/us-central1/customJobs/8437305747951321088')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/8437305747951321088?project=65830871515
CustomJob projects/65830871515/locations/us-central1/customJobs/8437305747951321088 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/65830871515/locations/us-central1/customJobs/8437305747951321088 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/65830871515/locations/us-central1/customJobs/8437305747951321088 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/65830871515/locations/us-central1/customJobs/8437305747951321088 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/65830871515/locations/us-central1/customJobs/843730574795132

RuntimeError: Job failed with:
code: 3
message: "The replica workerpool0-0 exited with a non-zero status of 1. \nTraceback (most recent call last):\n  File \"/opt/conda/lib/python3.10/runpy.py\", line 196, in _run_module_as_main\n    return _run_code(code, main_globals, None,\n  File \"/opt/conda/lib/python3.10/runpy.py\", line 86, in _run_code\n    exec(code, run_globals)\n  File \"/root/.local/lib/python3.10/site-packages/trainer/task.py\", line 67, in <module>\n    global_noise_data = torch.zeros([512, 3, 224, 224]).cuda()\n  File \"/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py\", line 229, in _lazy_init\n    torch._C._cuda_init()\nRuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx\n\nTo find out more about why your job exited please check the logs: https://console.cloud.google.com/logs/viewer?project=65830871515&resource=ml_job%2Fjob_id%2F843730574795132